In [ ]:
# # connect google drive
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import library

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

# Part1: goodFeaturesToTrack
- Fill the missing part (denoted as ```fill here```) of the code
- We provide procedure comments for complete the function

In [104]:
def goodFeaturesToTrack(image, maxCorners=100, qualityLevel=0.03, blocksize=7):

    # Image bluring wih averaging filter
    # Only cv2.filter2D is allowed for convolution operation!
    average_filter = np.ones((blocksize, blocksize), np.float32)/(blocksize*blocksize)
    # gaussian_filter = np.array([[1/16, 2/16, 1/16], [2/16, 4/16, 2/16], [1/16, 2/16, 1/16]])
    image = cv2.filter2D(image, -1, average_filter) / 255.0

    # Compute gradients
    Ix = cv2.filter2D(image, -1, np.array([[-1/2, 0, 1/2]]))
    Iy = cv2.filter2D(image, -1, np.array([[-1/2], [0], [1/2]]))

    # Compute products of gradients at each pixel
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Compute the sums of products of gradients in local windows
    Sxx = cv2.filter2D(Ixx, -1, np.ones((3, 3)))[1:-1, 1:-1]
    Syy = cv2.filter2D(Iyy, -1, np.ones((3, 3)))[1:-1, 1:-1]
    Sxy = cv2.filter2D(Ixy, -1, np.ones((3, 3)))[1:-1, 1:-1]

    # Compute the determinant and trace of the matrix M for each pixel
    detM = Sxx * Syy - Sxy**2
    traceM = Sxx + Syy

    # Compute the Harris response with detM and traceM
    harris_response = detM - qualityLevel * (traceM**2)

    # Threshold the Harris response to find candidate corners
    corners = np.argwhere(harris_response >= 0.1 * np.max(harris_response))

    # Sort the corners by Harris response in descending order
    sorted_corners = corners[np.argsort(harris_response[corners[:, 0], corners[:, 1]])[::-1]]

    # Keep the top 'maxCorners' corners
    selected_corners = sorted_corners[:maxCorners]

    final_corners = np.array(selected_corners) + 1
    final_corners = final_corners.reshape(-1, 1, 2)

    return final_corners

# Part2: Optical flow with Lukas-Kanade
- Fill the missing part (denoted as ```fill here```) of the code
- We provide procedure comments for complete the function

In [105]:

def optical_flow(old_frame, new_frame, window_size, min_quality):

    feature_list = goodFeaturesToTrack(old_frame, max_corners, min_quality, blocksize)

    w = int(window_size/2)

    # Normalize
    old_frame = old_frame / 255
    new_frame = new_frame / 255

    # Convolve to get gradients w.r.to X, Y and T dimensions
    kernel_x = np.array([[-1/2, 0, 1/2]])
    kernel_y = np.array([[-1/2], [0], [1/2]])
    kernel_t = np.array([[1]])

    # cv2.filter2D is allowed for convolution!
    fx =  cv2.filter2D(old_frame, -1, kernel_x)
    fy =  cv2.filter2D(old_frame, -1, kernel_y)
    ft =  cv2.filter2D(new_frame - old_frame, -1, kernel_t)

    u = np.zeros(old_frame.shape)
    v = np.zeros(old_frame.shape)

    for feature in feature_list:  # for every corner
        i, j = feature.ravel()  # get cordinates of the corners (i,j).
        i, j = int(i), int(j)  # i,j are floats initially so convert to integer type

        I_x = fx[i-w:i+w+1, j-w:j+w+1].ravel()
        I_y = fy[i-w:i+w+1, j-w:j+w+1].ravel()
        I_t = ft[i-w:i+w+1, j-w:j+w+1].ravel()

        b = np.reshape(I_t, (I_t.shape[0], 1))      # b : (225, 1)  225가 아닌 195 or 210 인 경우도 존재
        A = np.vstack((I_x, I_y)).T                 # A : (225, 2)  225가 아닌 195 or 210 인 경우도 존재

        weight_size = A.shape[0]
        weight_vec = np.identity(weight_size) / 2
        if weight_size // 2:
            # weight_vec[int(weight_size/2)-int(weight_size/4):int(weight_size/2)-int(weight_size/4)+1,
            #            int(weight_size/2)-int(weight_size/4):int(weight_size/2)-int(weight_size/4)+1] = 0.5
            weight_vec[int(weight_size/2),int(weight_size/2)] = 1.0
        else:
            # weight_vec[int(weight_size/2)-int(weight_size/4):int(weight_size/2)-int(weight_size/4)+1,
            #            int(weight_size/2)-int(weight_size/4):int(weight_size/2)-int(weight_size/4)+1] = 0.5
            weight_vec[int(weight_size/2)-1:int(weight_size/2)+1,int(weight_size/2)-1:int(weight_size/2)+1] = 1.0
        Weight = weight_vec   # Weight : (A.shape[0], A.shape[0])

        # U = (np.linalg.pinv(A.T @ A)) @ (A.T @ b)  # Solving for (u,v) i.e., U
        U = (np.linalg.pinv(A.T @ Weight @ A)) @ (A.T @ Weight @ b)

        u[i, j] = U[0][0]
        v[i, j] = U[1][0]

    return (u, v)


# Main function
- If part1 and part2 were filled properly, the 'output.avi' will be generated!
- For google colab, as cv2.imshow() is not provided, so please use cv2_imshow (google.colab.patches) instead  

In [106]:
# cap = cv2.VideoCapture('/content/drive/MyDrive/CV_project/optical_flow/slow.mp4')
cap = cv2.VideoCapture('C:/Users/jeong/Desktop/3_2학기/Computer_vision/CV_project/optical_flow/slow.mp4')

# Take first frame and find corners in it
ret, old_frame = cap.read()

# Width and height of the file to save
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# 'output.mp4' will be generated!
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4',  fourcc, 30.0, (int(width), int(height)))

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Shi Tomasi parameter
max_corners = 100
min_quality = 0.05      # initial : 0.03
blocksize = 7
p0 = goodFeaturesToTrack(old_frame, max_corners, min_quality, blocksize)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, current_frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    
    # calculate optical flow
    U, V = optical_flow(old_gray, frame_gray, 15, min_quality)

    for i in range(current_frame.shape[0]):
        for j in range(current_frame.shape[1]):
            u, v = U[i][j], V[i][j]
            if u and v:
                mask = cv2.line(mask, (j, i), (int(round(j + u)), int(round(i + v))), (0, 255, 0), 2)
                frame = cv2.arrowedLine(current_frame, (j, i), (int(round(j + u)), int(round(i + v))), (0, 255, 0), thickness=2)
                current_frame = cv2.add(current_frame, mask)

    # Display the frame with optical flow vectors
    cv2.imshow("frame", current_frame)
    # cv2.imshow("gray", frame_gray)
    out.write(current_frame)
    # Break the loop if 'Esc' key is pressed
    if cv2.waitKey(30) == 27:
        break

    # Set the current frame as the previous frame for the next iteration
    old_gray = frame_gray

# Release the video capture object
cap.release()
out.release()

# Close the plot window when done
plt.close()
cv2.destroyAllWindows()